In [9]:
%matplotlib inline
from __future__ import division

import pandas as pd
import matplotlib.pyplot as plt
import utils
import datetime

In [2]:
visitors, events, devices, url_categories = utils.load_data(event_categories=True)

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5,6,7,8,11,12,13,14,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
len(events[~events.site_category.isnull()])/len(events)

0.9838107265260091

In [4]:
sample_npis = list(visitors[(visitors.professional_designation=='Physician')\
                       &(~visitors.npi_number.isnull())\
                       &(visitors.exclude==0)].npi_number.sample(10000))

In [27]:
def categorize_frequency(n):
    if n < 1:
        return 'Less than 1 visit a month' 
    elif n <= 10:
        return '1-10 times per month'
    else:
        return 'More than 10x per month'
    
def get_quarters(dt):
    if dt < datetime.date(2017, 5, 1):
        return 'q1'
    elif dt < datetime.date(2017, 8, 1):
        return 'q2'
    
def npi_level_summary_statistics(output_file=None, npis=None):
    # if given a list of users, pull in those. otherwise, use all available data
    if npis:
        v = visitors[visitors.npi_number.isin(npis)]
    else:
        v = visitors[visitors.exclude==0]
    ev = pd.merge(v, events, on='dg_id')
    if len(ev) == 0:
        print 'target list has no web activity'
        return
    # check which users have web activity
    event_dgids = set(ev.dg_id)
    v['has_event'] = v.dg_id.apply(lambda x: 1 if x in event_dgids else 0)
    
    # check frequency of web activity (timeframe denominator starts with the earliest date the user has activity)
    max_dt = max(events.timestamp).date()
    ev['dt'] = ev.timestamp.apply(lambda x: x.date())
    ev['quarter'] = ev.dt.apply(lambda x: get_quarters(x))
    freq = ev.drop_duplicates(subset=['dg_id', 'dt']).groupby(['dg_id']).dt.agg(['min', 'max', 'count']).reset_index()
    freq['sessions_per_month'] = freq.apply(lambda x: x['count']/(max(1, (max_dt-x['min']).days)/30), axis=1)
    freq['frequency_category'] = freq.sessions_per_month.apply(lambda x: categorize_frequency(x))
    v = pd.merge(v, freq[['dg_id', 'sessions_per_month', 'frequency_category']], how='left')
    
    # separate frequency by quarter
    freqq = ev.drop_duplicates(subset=['dg_id', 'dt']).groupby(['dg_id', 'quarter']).dt.agg(['min', 'max', 'count']).reset_index()
    freqq['sessions_per_month'] = freqq.apply(lambda x: x['count']/(max(1, (max_dt-x['min']).days)/30), axis=1)
    freqq['frequency_category'] = freqq.sessions_per_month.apply(lambda x: categorize_frequency(x))
    freqq = freqq.pivot(index='dg_id', columns='quarter', values='frequency_category').reset_index()
    freqq.columns= ['dg_id', 'frequency_category_q1', 'frequency_category_q2']
    v = pd.merge(v, freqq, on='dg_id', how='left')
    
    # grab avg number of urls per session
    urls_per_session = ev.groupby(['dg_id', 'session_id']).url.nunique().reset_index()\
        .groupby('dg_id').url.mean().reset_index()
    urls_per_session = urls_per_session.rename(columns={'url':'urls_per_session'})
    v = pd.merge(v, urls_per_session, on='dg_id', how='left')
    
    tableau_columns = [
        'dg_id', 
        'identity_type', 
        'professional_designation', 
        'npi_number', 
        'primary_specialty', 
        'primary_specialty_group', 
        'birth_year', 
        'grad_year', 
        'gender', 
        'has_event', 
        'sessions_per_month', 
        'frequency_category',
        'frequency_category_q1',
        'frequency_category_q2',
        'urls_per_session' 
    ]
    
    if output_file:
        v[tableau_columns].to_csv(output_file)
    else:
        return v[tableau_columns]

In [28]:
npis = list(v[~v.npi_number.isnull()].npi_number.sample(5))

In [29]:
npi_level_summary_statistics(npis=npis)

target list has no web activity


In [30]:
event_npis = list(visitors[(visitors.dg_id.isin(events.dg_id.sample(5)))&(~visitors.npi_number.isnull())].npi_number)

In [31]:
df= npi_level_summary_statistics(npis=event_npis)

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
df = npi_level_summary_statistics()

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [34]:
len(df)

5964081

In [36]:
df[df.has_event==1].sample(10)

,dg_id,identity_type,professional_designation,npi_number,primary_specialty,primary_specialty_group,birth_year,grad_year,gender,has_event,sessions_per_month,frequency_category,frequency_category_q1,frequency_category_q2,urls_per_session
2294723,U082272135,AUT,PA,1.467709e+09,Unspecified,Midlevel,0,0,F,1,0.789474,Less than 1 visit a month,None,Less than 1 visit a month,1.00
4580608,X3760100,POI,POI,NaN,NaN,NaN,0,0,NaN,1,2.926829,1-10 times per month,None,1-10 times per month,1.80
4417208,X361304,POI,POI,NaN,NaN,NaN,0,0,NaN,1,0.508475,Less than 1 visit a month,Less than 1 visit a month,None,1.50
739106,U080109108,AUT,Physician,1.124244e+09,Addiction Psychiatry (ADP),Psychiatry,1978,2002,M,1,0.236220,Less than 1 visit a month,Less than 1 visit a month,None,1.00
1402107,U080783854,AUT,Physician,1.093776e+09,Neurology (N),Neurology,1959,1985,M,1,0.720000,Less than 1 visit a month,Less than 1 visit a month,None,1.25
4651240,X3823670,POI,POI,NaN,NaN,NaN,0,0,NaN,1,0.937500,Less than 1 visit a month,None,Less than 1 visit a month,1.00
1368716,U080749893,AUT,Physician,1.124056e+09,Orthopedic Surgery (ORS),Orthopedics,1956,1984,M,1,0.206897,Less than 1 visit a month,Less than 1 visit a month,None,1.00
2113060,U082081235,AUT,NP,1.902921e+09,Pediatrics,Midlevel,0,0,F,1,1.153846,1-10 times per month,None,1-10 times per month,1.00
1721060,U081106998,AUT,Physician,1.780757e+09,Pediatrics (PD),Pediatrics,1949,1974,F,1,0.254237,Less than 1 visit a month,Less than 1 visit a month,None,2.00
2333350,U082316856,AUT,PA,1.215913e+09,Emergency Medicine,Midlevel,1960,0,F,1,0.468750,Less than 1 visit a month,None,Less than 1 visit a month,1.00
